In [30]:
################################################
# intialize glove
################################################
# >pip install glove_python
# >wget http://www.google.com/search?q=glove+word+embeddings+download+those+dope+pre+trained+vectors
# >unzip dope_glove_shit.zip
# >echo "set GLOVE_LOCATION to one of those files"
################################################

from glove import Glove

GLOVE_LOCATION = "../glove.6B/glove.6B.50d.txt"

glove_data = Glove.load_stanford(GLOVE_LOCATION)

print(glove_data.most_similar('test', number=10))

[('tests', 0.9164356610046378), ('testing', 0.81992214225170001), ('tested', 0.74428756870386181), ('final', 0.69102279899050822), ('taking', 0.68790020450861178), ('results', 0.68468909013686297), ('match', 0.67696818430673733), ('determine', 0.67679768970507126), ('challenge', 0.6747705556743957)]


In [36]:
################################################
# file parsing functions
################################################
from nltk.tokenize import TweetTokenizer
import string, re

# definitions
HC="HillaryClinton"
DT="realDonaldTrump"
NA="none"
HANDLES = [HC,DT,NA]
HANDLE_MAP = {NA:0, HC:1, DT:2}

# read csv file, return handles and tweets
def parse_tweet_csv(file, file_encoding="utf8"):
    # init
    handles, tweets = [], []
    
    # read file
    linenr = -1
    with open(file, encoding=file_encoding) as input:
        try:
            for line in input:
                linenr += 1
                if linenr == 0: continue
                
                # get contents
                line = line.split(",")
                if line[0] in HANDLES: #label and irst line of tweet
                    handles.append(line[0])
                    tweet = ','.join(line[1:])
                    tweets.append(tweet)
                else: #second+ line of tweet
                    tweet = tweets.pop()
                    tweet += ','.join(line)
                    tweets.append(tweet)
        except Exception as e:
            print("Exception at line {}: {}".format(linenr, e))
            raise e
    
    # sanity checks
    assert len(handles) == len(tweets)
    print("Found {} tweets in {} lines".format(len(tweets), linenr + 1))
    
    # return data
    return handles, tweets


##########################################
### coverting tweet strings to numbers ###
#tokenize
_tokenizer = TweetTokenizer()
_punctuation = set(string.punctuation)
def tokenize(tweet, lowercase=True, strip_urls=True, strip_punctuation=True):
    tokens = _tokenizer.tokenize(tweet)
    if lowercase: tokens = list(map(lambda x: x.lower(), tokens))
    if strip_urls: tokens = list(filter(lambda x: not x.startswith("http"), tokens))
    if strip_punctuation: #https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        tokens = list(filter(lambda x: x.startswith(u'@') or x.startswith(u'#') or x not in _punctuation and not re.match(u"[^\w\d'\s$]+", x), tokens))
    return tokens
# glove
def get_glove_vector(tokens):
    def transform_paragraph(self, paragraph, epochs=50, ignore_missing=False):
        """
        Transform an iterable of tokens into its vector representation
        (a paragraph vector).
    
        Experimental. This will return something close to a tf-idf
        weighted average of constituent token vectors by fitting
        rare words (with low word bias values) more closely.
        """
        import collections
        import numpy as np
        import glove
        from glove.glove_cython import fit_vectors, transform_paragraph
        
    
        if self.word_vectors is None:
            raise Exception('Model must be fit to transform paragraphs')
    
        if self.dictionary is None:
            raise Exception('Dictionary must be provided to '
                            'transform paragraphs')
    
        cooccurrence = collections.defaultdict(lambda: 0.0)
    
        for token in paragraph:
            try:
                cooccurrence[self.dictionary[token]] += self.max_count / 10.0
            except KeyError:
                if not ignore_missing:
                    raise
    
        random_state = glove.glove.check_random_state(self.random_state)
    
        word_ids = np.array(list(cooccurrence.keys()), dtype=np.int32)
        values = np.array(list(cooccurrence.values()), dtype=np.float64)
        shuffle_indices = np.arange(len(word_ids), dtype=np.int32)
    
        # Initialize the vector to mean of constituent word vectors
        paragraph_vector = np.mean(self.word_vectors[word_ids], axis=0)
        sum_gradients = np.ones_like(paragraph_vector)
    
        # Shuffle the coocurrence matrix
        random_state.shuffle(shuffle_indices)
        transform_paragraph(self.word_vectors,
                            self.word_biases,
                            paragraph_vector,
                            sum_gradients,
                            word_ids,
                            values,
                            shuffle_indices,
                            self.learning_rate,
                            self.max_count,
                            self.alpha,
                            epochs)
    
        return paragraph_vector
    
    return transform_paragraph(glove_data, " ".join(tokens), ignore_missing=True)
# handle
def import_text(tweets):
    return [tokenize(tweet) for tweet in tweets]


def int_labels(labels):
    return list(map(lambda x: HANDLE_MAP[x], labels))
    

handles, tweets = parse_tweet_csv("train.csv")
handles = int_labels(handles)
# tweets = import_text(tweets)

### for validation
for tweet in tweets[0:16]:
    print(tokenize(tweet))
    print(get_glove_vector(tokenize(tweet)))
    print()
# for handle in int_labels(handles[0:7]):
#     print(handle)

Found 4743 tweets in 6251 lines
['the', 'question', 'in', 'this', 'election', 'who', 'can', 'put', 'the', 'plans', 'into', 'action', 'that', 'will', 'make', 'your', 'life', 'better']
[-0.02025492  0.17447466  0.1113957   0.10850239 -0.15766437 -0.00880653
  0.08078878 -0.18767856 -0.01001138 -0.02274603  0.06846685 -0.0201696
  0.18227073  0.02041927 -0.00273995  0.05156851  0.02180317  0.07600065
  0.23698627  0.1188742   0.27986816 -0.04159186  0.19940441 -0.03171541
  0.02606152  0.07798278 -0.05114254  0.11331858 -0.01473179  0.02611706
  1.34425389  0.31747192 -0.00419791 -0.00589631 -0.20707862 -0.20163681
 -0.00490502 -0.09823382 -0.32663391  0.01303974  0.17537923 -0.26139659
 -0.03909616 -0.06620144  0.26834694 -0.19813136 -0.38873964  0.01047187
 -0.03145515  0.24641894]

['last', 'night', 'donald', 'trump', 'said', 'not', 'paying', 'taxes', 'was', 'smart', 'you', 'know', 'what', 'i', 'call', 'it', 'unpatriotic']
[ -8.85836716e-02   2.28314284e-01   1.39221142e-01   1.3345037